# Implement GLM and Contrast Function
A generalized linear model is used to find where brain activity changes given different tasks
The contrast function compares the brain activity between target and general task.

We are interested in the following contrast:
- Differentiate brain region activity between target and general task during audio condition
- Differentiate brain region activity between target and general task during visual condition
- Differentiate brain region activity between audio and visual condition after contrast is applied to both conditions

In [70]:
# import dependencies
from nilearn.glm.first_level import FirstLevelModel
import numpy as np
import nibabel as nib
import nilearn.masking as masking


In [71]:
# Implement glm base model
def fit_glm(fmri_img, events_data, tr):
    """ 
    Fit a glm to the fmri data
    """
    print("fMRI data shape: ", fmri_img.shape)
    print("fMRI data min/max: ", np.min(fmri_img.get_fdata()), np.max(fmri_img.get_fdata()))

    model = FirstLevelModel(
        t_r=tr,
        mask_img = fmri_img,
        standardize = True,
    )
    
    glm = model.fit(fmri_img, events = events_data)
    return glm

In [72]:
# Categorize stimuli into target and general task
def categorize_stimuli(events_data, data_path):
    """ 
    Categorize stimuli based on the filename patterns
    For visual data: 'inh' = general task, 'sel' = target task
    For audio data: NA
    events_data: pandas dataframe pulled from the events.tsv file
    data_path: path to the visual folder 
    """
    if 'visual' in data_path: # if there is a data file with visual in the path
        if 'inh' in data_path:
            events_data['condition'] = 'general' # inhibition task
        elif 'sel' in data_path:
            events_data['condition'] = 'target' # selection task

    #TODO: Need to add audio data categorization
    
    return events_data

In [73]:
# Implement contrast function
def contrast_glm_stimuli(glm, events_data, file_path, contrast_def = {'target': 1, 'general': -1}): 
    """ 
    Apply a contrast to the glm results 
    """
    events = categorize_stimuli(events_data, file_path)
    contrast = glm.compute_contrast(contrast_def)
    return contrast

def contrast_glm_conditions(glm, contrast_def = {'audio': 1, 'visual': -1}):
    """ 
    Apply a contrast to the glm results
    """
    contrast = glm.compute_contrast(contrast_def)
    return contrast

In [74]:
import os
import glob
import pandas as pd
from nilearn.image import mean_img
# Run the glm and contrast
def run_glm_contrast(fmri_img, data_path, tr):
    """ 
    Run the glm on the fmri data and apply contrast
    """
    fmri_3d = mean_img(fmri_img)
    print("fMRI 3D data shape: ", fmri_3d.shape)

    # get all events files in the events_data_path
    pattern = os.path.join(data_path, 'sub-*/func/*events.tsv')
    events_files = glob.glob(pattern, recursive = True)

    if not events_files:
        raise ValueError(f"No events files found in {pattern}")
    
    # read the first events file 
    events_data = pd.read_csv(events_files[0], sep = '\t')

    # fit the glm
    glm = fit_glm(fmri_3d, events_data, tr)

    # apply contrast
    contrast_stimuli = contrast_glm_stimuli(glm, events_data)
    contrast_conditions = contrast_glm_conditions(glm)
    return contrast_stimuli, contrast_conditions

data_path = "../data/visual"
fmri_data = "../results/visual/cleaned_data_visual.nii.gz" #TODO: need to add audio data
fmri_img = nib.load(fmri_data)
tr = 1.5 # test with visual data
run_glm_contrast(fmri_img, data_path, tr)

fMRI 3D data shape:  (64, 64, 32)
fMRI data shape:  (64, 64, 32)
fMRI data min/max:  -3.631302388384938e-05 3.73113865722329e-05


ValueError: Given mask is not made of 2 values: [-3.63130239e-05 -3.18509384e-05 -2.96198957e-05 -2.73888529e-05
 -2.51578102e-05 -2.29267674e-05 -2.06957247e-05 -1.84646820e-05
 -1.62336392e-05 -1.40025965e-05 -1.17715537e-05 -9.54051099e-06
 -7.30946825e-06 -5.07842551e-06 -2.84738277e-06 -6.16340026e-07
  1.61470271e-06  3.84574546e-06  6.07678820e-06  8.30783094e-06
  1.05388737e-05  1.27699164e-05  1.50009592e-05  1.72320019e-05
  1.94630446e-05  2.16940874e-05  2.39251301e-05  2.61561729e-05
  2.83872156e-05  3.06182583e-05  3.28493011e-05  3.50803438e-05
  3.73113866e-05]. Cannot interpret as true or false.